In [6]:
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
import numpy as np
import json
import random
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import Tensor, nn
from torch.overrides import has_torch_function, handle_torch_function
from typing import Optional, Tuple
import warnings
from sksurv.metrics import concordance_index_censored
import torch.nn.parallel

In [21]:
class CoAttnData(Dataset):
    def __init__(self, task="KIRC", num_bins=4, fold=0, mode="train", settings="PaRa_MIL"):
        super(CoAttnData, self).__init__()

        # dataset split
        json_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention_prognosis/splits/json_{task.lower()}/"
        self.fold = fold
        with open(os.path.join(json_path, f"fold_{self.fold}.json"), "r") as f:
            self.patients = json.load(f)[mode]

        # get discrete survival label
        self.bins = num_bins
        self.label_path = "/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention_prognosis/preparation/" \
                          f"clinical_info_clean_{task.lower()}.csv"
        self.labels = pd.read_csv(self.label_path, index_col="patient")

        # define pathomics path generated by pretraining graph
        self.setting = settings
        self.pathomics_para_mil_base = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TCGA-{task}/patch_features_merged_resnet/"
        self.pathomics_para_mil_wo_base = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TCGA-{task}/patch_features_resnet/"

        # define radiomics path generated by PyRadiomics
        radiomics_base = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TRAD-{task}/summary_radiomics.csv"
        radiomics_data = pd.read_csv(radiomics_base).set_index("imageFile").loc[self.patients, :]
        all_columns = list(radiomics_data.columns)
        diagnostic_cols = sorted(list(filter(lambda x: "diagnostics" in x, all_columns)))
        shape_cols = sorted(list(filter(lambda x: "original_shape" in x, all_columns)))
        order_cols = sorted(list(filter(lambda x: "original_firstorder" in x, all_columns)))
        texture_cols = sorted(list(filter(lambda x: ("original_glcm" in x) or ("original_gldm" in x)
                                          or ("original_glrlm" in x) or ("original_glszm" in x)
                                          or ("original_ngtdm" in x), all_columns)))
        log_cols = sorted(list(filter(lambda x: "log-sigma" in x, all_columns)))
        wavelet_cols = sorted(list(filter(lambda x: "wavelet" in x, all_columns)))
        assert len(all_columns) == len(diagnostic_cols) + len(shape_cols) + len(order_cols) \
               + len(texture_cols) + len(log_cols) + len(wavelet_cols)

        # perform ss for radiomics data
        radiomics_data = radiomics_data.drop(diagnostic_cols, axis=1)
        ss = StandardScaler()
        radio_data = pd.DataFrame(ss.fit_transform(radiomics_data),
                                  index=radiomics_data.index,
                                  columns=radiomics_data.columns)
        self.shape = radio_data[shape_cols]
        self.order = radio_data[order_cols]
        self.texture = radio_data[texture_cols]
        self.log_sigma = radio_data[log_cols]
        self.wavelet = radio_data[wavelet_cols]

        # define radiomics path generated by RadFM
        self.radio_cnn_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TRAD-{task}/ct_nii_features/"

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, item):
        patient_name = self.patients[item]

        labels = self.labels.loc[patient_name]
        surv_discrete = labels["time_bins"]
        surv_time = labels["time"]
        censor = 1 - labels["event"]

        # pathological data
        if self.setting == "PaRa_MIL_wo":
            patho = torch.load(os.path.join(self.pathomics_para_mil_wo_base, f"{patient_name}.pt")).to(torch.float32)
        else:
            patho = torch.load(os.path.join(self.pathomics_para_mil_base, f"{patient_name}.pt")).to(torch.float32)

        # radiological data
        radio_shape = torch.from_numpy(self.shape.loc[patient_name, :].values).to(torch.float32)
        radio_order = torch.from_numpy(self.order.loc[patient_name, :].values).to(torch.float32)
        radio_texture = torch.from_numpy(self.texture.loc[patient_name, :].values).to(torch.float32)
        radio_log_sigma = torch.from_numpy(self.log_sigma.loc[patient_name, :].values).to(torch.float32)
        radio_wavelet = torch.from_numpy(self.wavelet.loc[patient_name, :].values).to(torch.float32)

        # RadFM data
        radio_cnn = torch.load(os.path.join(self.radio_cnn_path, f"{patient_name}.pt")).squeeze(0).to(torch.float32)

        data = [patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn,
                surv_discrete, surv_time, censor, patient_name]

        return data

In [8]:

def SNN_Block(dim1, dim2, dropout=0.25):
    r"""
    Multilayer Reception Block w/ Self-Normalization (Linear + ELU + Alpha Dropout)

    args:
        dim1 (int): Dimension of input features
        dim2 (int): Dimension of output features
        dropout (float): Dropout rate
    """
    import torch.nn as nn

    return nn.Sequential(
        nn.Linear(dim1, dim2),
        nn.ELU(),
        nn.AlphaDropout(p=dropout, inplace=False))


class Attn_Net_Gated(nn.Module):
    def __init__(self, L=1024, D=256, dropout=False, n_classes=1):
        r"""
        Attention Network with Sigmoid Gating (3 fc layers)

        args:
            L (int): input feature dimension
            D (int): hidden layer dimension
            dropout (bool): whether to apply dropout (p = 0.25)
            n_classes (int): number of classes
        """
        super(Attn_Net_Gated, self).__init__()
        self.attention_a = [
            nn.Linear(L, D),
            nn.Tanh()]

        self.attention_b = [nn.Linear(L, D), nn.Sigmoid()]
        if dropout:
            self.attention_a.append(nn.Dropout(0.25))
            self.attention_b.append(nn.Dropout(0.25))

        self.attention_a = nn.Sequential(*self.attention_a)
        self.attention_b = nn.Sequential(*self.attention_b)
        self.attention_c = nn.Linear(D, n_classes)

    def forward(self, x):
        a = self.attention_a(x)
        b = self.attention_b(x)
        A = a.mul(b)
        A = self.attention_c(A)  # N x n_classes
        return A, x

In [9]:
class PaRaMIL(nn.Module):
    def __init__(self, n_classes=4, dropout=0.25):
        super(PaRaMIL, self).__init__()

        self.omic_sizes = [14, 18, 75, 279, 744, 768]  # radiomics sizes
        self.n_classes = n_classes

        # pathomics bag formulation
        self.wsi_net = nn.Sequential(*[nn.Linear(512, 192),
                                       nn.ReLU(),
                                       nn.Dropout(0.25)])
        # radiomics bag formulation
        sig_networks = []
        for input_dim in self.omic_sizes:
            sig_networks.append(nn.Sequential(*[SNN_Block(dim1=input_dim, dim2=256),
                                                SNN_Block(dim1=256, dim2=256, dropout=0.25)]))
        self.sig_networks = nn.ModuleList(sig_networks)

        # coattention module
        size_embed = 256
        self.coattn = MultiheadAttention(embed_dim=size_embed, num_heads=1)

        # MIL module
        path_encoder_layer = nn.TransformerEncoderLayer(d_model=size_embed, nhead=8, dim_feedforward=512,
                                                        dropout=dropout, activation='relu')
        self.path_transformer = nn.TransformerEncoder(path_encoder_layer, num_layers=2)
        self.path_attention_head = Attn_Net_Gated(L=size_embed, D=size_embed,
                                                  dropout=dropout, n_classes=1)
        self.path_rho = nn.Sequential(*[nn.Linear(size_embed, size_embed),
                                        nn.ReLU(), nn.Dropout(dropout)])

        omic_encoder_layer = nn.TransformerEncoderLayer(d_model=size_embed, nhead=8, dim_feedforward=512,
                                                        dropout=dropout, activation='relu')
        self.omic_transformer = nn.TransformerEncoder(omic_encoder_layer, num_layers=2)
        self.omic_attention_head = Attn_Net_Gated(L=size_embed, D=size_embed,
                                                  dropout=dropout, n_classes=1)
        self.omic_rho = nn.Sequential(*[nn.Linear(size_embed, size_embed),
                                        nn.ReLU(), nn.Dropout(dropout)])

        self.mm = nn.Sequential(*[nn.Linear(size_embed * 2, size_embed), nn.ReLU(),
                                  nn.Linear(size_embed, size_embed),
                                  nn.ReLU()])
        self.classifier = nn.Linear(size_embed, n_classes)

    def forward(self, **kwargs):
        x_path = kwargs['x_path']
        x_omic = [kwargs['x_omic%d' % i] for i in range(1, 7)]

        h_path_bag = self.wsi_net(x_path[:, :, :512])
        h_omic = [self.sig_networks[idx].forward(sig_feat) for idx, sig_feat in enumerate(x_omic)]
        h_omic_bag = torch.stack(h_omic)
        h_path_bag = torch.concat([h_path_bag, x_path[:, :, 512:]], dim=2)

        h_path_coattn, A_coattn = self.coattn(h_omic_bag, h_path_bag, h_path_bag)

        h_path_trans = self.path_transformer(h_path_coattn)
        A_path, h_path = self.path_attention_head(h_path_trans.squeeze(1))
        A_path = torch.transpose(A_path, 1, 0)
        h_path = torch.mm(F.softmax(A_path, dim=1), h_path)
        h_path = self.path_rho(h_path).squeeze()

        h_omic_trans = self.omic_transformer(h_omic_bag)
        A_omic, h_omic = self.omic_attention_head(h_omic_trans.squeeze(1))
        A_omic = torch.transpose(A_omic, 1, 0)
        h_omic = torch.mm(F.softmax(A_omic, dim=1), h_omic)
        h_omic = self.omic_rho(h_omic).squeeze()

        h = self.mm(torch.cat([h_path, h_omic], dim=0))

        logits = self.classifier(h).unsqueeze(0)
        Y_hat = torch.topk(logits, 1, dim=1)[1]
        hazards = torch.sigmoid(logits)
        S = torch.cumprod(1 - hazards, dim=1)

        attention_scores = {'coattn': A_coattn, 'path': A_path, 'omic': A_omic}

        return hazards, S, Y_hat, attention_scores


###
# ========== Modifying PyTorch Functionalities ======================
###
from torch.nn.functional import *


def multi_head_attention_forward(
        query: Tensor,
        key: Tensor,
        value: Tensor,
        embed_dim_to_check: int,
        num_heads: int,
        in_proj_weight: Tensor,
        in_proj_bias: Tensor,
        bias_k: Optional[Tensor],
        bias_v: Optional[Tensor],
        add_zero_attn: bool,
        dropout_p: float,
        out_proj_weight: Tensor,
        out_proj_bias: Tensor,
        training: bool = True,
        key_padding_mask: Optional[Tensor] = None,
        need_weights: bool = True,
        need_raw: bool = True,
        attn_mask: Optional[Tensor] = None,
        use_separate_proj_weight: bool = False,
        q_proj_weight: Optional[Tensor] = None,
        k_proj_weight: Optional[Tensor] = None,
        v_proj_weight: Optional[Tensor] = None,
        static_k: Optional[Tensor] = None,
        static_v: Optional[Tensor] = None,
):
    r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        embed_dim_to_check: total dimension of the model.
        num_heads: parallel attention heads.
        in_proj_weight, in_proj_bias: input projection weight and bias.
        bias_k, bias_v: bias of the key and value sequences to be added at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        dropout_p: probability of an element to be zeroed.
        out_proj_weight, out_proj_bias: the output projection weight and bias.
        training: apply dropout if is ``True``.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.
        use_separate_proj_weight: the function accept the proj. weights for query, key,
            and value in different forms. If false, in_proj_weight will be used, which is
            a combination of q_proj_weight, k_proj_weight, v_proj_weight.
        q_proj_weight, k_proj_weight, v_proj_weight, in_proj_bias: input projection weight and bias.
        static_k, static_v: static key and value used for attention operators.
    Shape:
        Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the zero positions
          will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensures that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.
        - static_k: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        - static_v: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
    """
    tens_ops = (query, key, value, in_proj_weight, in_proj_bias, bias_k, bias_v, out_proj_weight, out_proj_bias)
    if has_torch_function(tens_ops):
        return handle_torch_function(
            multi_head_attention_forward,
            tens_ops,
            query,
            key,
            value,
            embed_dim_to_check,
            num_heads,
            in_proj_weight,
            in_proj_bias,
            bias_k,
            bias_v,
            add_zero_attn,
            dropout_p,
            out_proj_weight,
            out_proj_bias,
            training=training,
            key_padding_mask=key_padding_mask,
            need_weights=need_weights,
            need_raw=need_raw,
            attn_mask=attn_mask,
            use_separate_proj_weight=use_separate_proj_weight,
            q_proj_weight=q_proj_weight,
            k_proj_weight=k_proj_weight,
            v_proj_weight=v_proj_weight,
            static_k=static_k,
            static_v=static_v,
        )
    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == embed_dim_to_check
    # allow MHA to have different sizes for the feature dimension
    assert key.size(0) == value.size(0) and key.size(1) == value.size(1)

    head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
    scaling = float(head_dim) ** -0.5

    if not use_separate_proj_weight:
        if (query is key or torch.equal(query, key)) and (key is value or torch.equal(key, value)):
            # self-attention
            q, k, v = linear(query, in_proj_weight, in_proj_bias).chunk(3, dim=-1)

        elif key is value or torch.equal(key, value):
            # encoder-decoder attention
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            if key is None:
                assert value is None
                k = None
                v = None
            else:

                # This is inline in_proj function with in_proj_weight and in_proj_bias
                _b = in_proj_bias
                _start = embed_dim
                _end = None
                _w = in_proj_weight[_start:, :]
                if _b is not None:
                    _b = _b[_start:]
                k, v = linear(key, _w, _b).chunk(2, dim=-1)

        else:
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim
            _end = embed_dim * 2
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            k = linear(key, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim * 2
            _end = None
            _w = in_proj_weight[_start:, :]
            if _b is not None:
                _b = _b[_start:]
            v = linear(value, _w, _b)
    else:
        q_proj_weight_non_opt = torch.jit._unwrap_optional(q_proj_weight)
        len1, len2 = q_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == query.size(-1)

        k_proj_weight_non_opt = torch.jit._unwrap_optional(k_proj_weight)
        len1, len2 = k_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == key.size(-1)

        v_proj_weight_non_opt = torch.jit._unwrap_optional(v_proj_weight)
        len1, len2 = v_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == value.size(-1)

        if in_proj_bias is not None:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias[0:embed_dim])
            k = linear(key, k_proj_weight_non_opt, in_proj_bias[embed_dim: (embed_dim * 2)])
            v = linear(value, v_proj_weight_non_opt, in_proj_bias[(embed_dim * 2):])
        else:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias)
            k = linear(key, k_proj_weight_non_opt, in_proj_bias)
            v = linear(value, v_proj_weight_non_opt, in_proj_bias)
    q = q * scaling

    if attn_mask is not None:
        assert (
                attn_mask.dtype == torch.float32
                or attn_mask.dtype == torch.float64
                or attn_mask.dtype == torch.float16
                or attn_mask.dtype == torch.uint8
                or attn_mask.dtype == torch.bool
        ), "Only float, byte, and bool types are supported for attn_mask, not {}".format(attn_mask.dtype)
        if attn_mask.dtype == torch.uint8:
            warnings.warn("Byte tensor for attn_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
            attn_mask = attn_mask.to(torch.bool)

        if attn_mask.dim() == 2:
            attn_mask = attn_mask.unsqueeze(0)
            if list(attn_mask.size()) != [1, query.size(0), key.size(0)]:
                raise RuntimeError("The size of the 2D attn_mask is not correct.")
        elif attn_mask.dim() == 3:
            if list(attn_mask.size()) != [bsz * num_heads, query.size(0), key.size(0)]:
                raise RuntimeError("The size of the 3D attn_mask is not correct.")
        else:
            raise RuntimeError("attn_mask's dimension {} is not supported".format(attn_mask.dim()))
        # attn_mask's dim is 3 now.

    # convert ByteTensor key_padding_mask to bool
    if key_padding_mask is not None and key_padding_mask.dtype == torch.uint8:
        warnings.warn(
            "Byte tensor for key_padding_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead."
        )
        key_padding_mask = key_padding_mask.to(torch.bool)

    if bias_k is not None and bias_v is not None:
        if static_k is None and static_v is None:
            k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = pad(attn_mask, (0, 1))
            if key_padding_mask is not None:
                key_padding_mask = pad(key_padding_mask, (0, 1))
        else:
            assert static_k is None, "bias cannot be added to static key."
            assert static_v is None, "bias cannot be added to static value."
    else:
        assert bias_k is None
        assert bias_v is None

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if k is not None:
        k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if v is not None:
        v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)

    if static_k is not None:
        assert static_k.size(0) == bsz * num_heads
        assert static_k.size(2) == head_dim
        k = static_k

    if static_v is not None:
        assert static_v.size(0) == bsz * num_heads
        assert static_v.size(2) == head_dim
        v = static_v

    src_len = k.size(1)

    if key_padding_mask is not None:
        assert key_padding_mask.size(0) == bsz
        assert key_padding_mask.size(1) == src_len

    if add_zero_attn:
        src_len += 1
        k = torch.cat([k, torch.zeros((k.size(0), 1) + k.size()[2:], dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros((v.size(0), 1) + v.size()[2:], dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = pad(attn_mask, (0, 1))
        if key_padding_mask is not None:
            key_padding_mask = pad(key_padding_mask, (0, 1))

    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    assert list(attn_output_weights.size()) == [bsz * num_heads, tgt_len, src_len]

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_output_weights.masked_fill_(attn_mask, float("-inf"))
        else:
            attn_output_weights += attn_mask

    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),
            float("-inf"),
        )
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len, src_len)

    attn_output_weights_raw = attn_output_weights
    attn_output_weights = softmax(attn_output_weights, dim=-1)
    attn_output_weights = dropout(attn_output_weights, p=dropout_p, training=training)

    attn_output = torch.bmm(attn_output_weights, v)
    assert list(attn_output.size()) == [bsz * num_heads, tgt_len, head_dim]
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = linear(attn_output, out_proj_weight, out_proj_bias)

    if need_weights:
        if need_raw:

            attn_output_weights_raw = attn_output_weights_raw.view(bsz, num_heads, tgt_len, src_len)
            return attn_output, attn_output_weights_raw

            # attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
            # return attn_output, attn_output_weights.sum(dim=1) / num_heads, attn_output_weights_raw, attn_output_weights_raw.sum(dim=1) / num_heads
        else:
            # average attention weights over heads
            attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
            return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None

# if float(torch.__version__.split('.')[0]) == 0 or (
#         float(torch.__version__.split('.')[0]) == 1 and float(torch.__version__.split('.')[1])) < 9:
#     from torch.nn.modules.linear import _LinearWithBias
# else:
#     from torch.nn.modules.linear import NonDynamicallyQuantizableLinear as _LinearWithBias
from torch.nn.modules.linear import NonDynamicallyQuantizableLinear as _LinearWithBias

from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
from torch.nn import Module


class MultiheadAttention(Module):
    r"""Allows the model to jointly attend to information
    from different representation subspaces.
    See reference: Attention Is All You Need

    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O
        \text{where} head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)

    Args:
        embed_dim: total dimension of the model.
        num_heads: parallel attention heads.
        dropout: a Dropout layer on attn_output_weights. Default: 0.0.
        bias: add bias as module parameter. Default: True.
        add_bias_kv: add bias to the key and value sequences at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        kdim: total number of features in key. Default: None.
        vdim: total number of features in value. Default: None.

        Note: if kdim and vdim are None, they will be set to embed_dim such that
        query, key, and value have the same number of features.

    Examples::

        >>> multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)
    """
    bias_k: Optional[torch.Tensor]
    bias_v: Optional[torch.Tensor]

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None,
                 vdim=None):
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.Tensor(embed_dim, embed_dim))
            self.k_proj_weight = Parameter(torch.Tensor(embed_dim, self.kdim))
            self.v_proj_weight = Parameter(torch.Tensor(embed_dim, self.vdim))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty(3 * embed_dim, embed_dim))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = _LinearWithBias(embed_dim, embed_dim)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty(1, 1, embed_dim))
            self.bias_v = Parameter(torch.empty(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def __setstate__(self, state):
        # Support loading old MultiheadAttention checkpoints generated by v1.1.0
        if '_qkv_same_embed_dim' not in state:
            state['_qkv_same_embed_dim'] = True

        super(MultiheadAttention, self).__setstate__(state)

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, need_raw=True, attn_mask=None):
        # type: (Tensor, Tensor, Tensor, Optional[Tensor], bool, Optional[Tensor]) -> Tuple[Tensor, Optional[Tensor]]
        r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. When given a binary mask and a value is True,
            the corresponding value on the attention layer will be ignored. When given
            a byte mask and a value is non-zero, the corresponding value on the attention
            layer will be ignored
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.

    Shape:
        - Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the position
          with the zero positions will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensure that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          is not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.

        - Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
        """
        if not self._qkv_same_embed_dim:
            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, need_raw=need_raw,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, need_raw=need_raw,
                attn_mask=attn_mask)

In [10]:
def nll_loss(hazards, S, Y, c, alpha=0.4, eps=1e-7):
    batch_size = len(Y)
    Y = Y.view(batch_size, 1)  # ground truth bin, 1,2,...,k
    c = c.view(batch_size, 1).float()  # censorship status, 0 or 1
    if S is None:
        S = torch.cumprod(1 - hazards, dim=1)  # surival is cumulative product of 1 - hazards
    # without padding, S(0) = S[0], h(0) = h[0]
    S_padded = torch.cat([torch.ones_like(c), S], 1)  # S(-1) = 0, all patients are alive from (-inf, 0) by definition
    # after padding, S(0) = S[1], S(1) = S[2], etc, h(0) = h[0]
    # h[y] = h(1)
    # S[1] = S(1)
    uncensored_loss = -(1 - c) * (
        torch.log(torch.gather(S_padded, 1, Y).clamp(min=eps)) + torch.log(torch.gather(hazards, 1, Y).clamp(min=eps))
    )
    censored_loss = -c * torch.log(torch.gather(S_padded, 1, Y + 1).clamp(min=eps))
    neg_l = censored_loss + uncensored_loss
    loss = (1 - alpha) * neg_l + alpha * uncensored_loss
    loss = loss.mean()
    return loss


class NLLSurvLoss(object):
    def __init__(self, alpha=0.15):
        self.alpha = alpha

    def __call__(self, hazards, S, Y, c, alpha=None):
        if alpha is None:
            return nll_loss(hazards, S, Y, c, alpha=self.alpha)
        else:
            return nll_loss(hazards, S, Y, c, alpha=alpha)

In [11]:
seed = 330
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [15]:
fold = 1
lr = 1e-4
decay = 2e-4
task = "SHRJ"
start_epoch = 0
max_epochs = 200
min_loss_val = 999
patience = 10
min_delta = 0.001
wait = 0
best_c_index_val = 0
gc = 16

In [23]:
# initialize dataset and put into dataloader
train_dataset = CoAttnData(fold=fold, task=task, mode="train", settings="PaRa_MIL")
val_dataset = CoAttnData(fold=fold, task=task, mode="val", settings="PaRa_MIL")
print(f"--- Fold {fold} ---")
print(f"Training on {len(train_dataset)} samples")
print(f"Validating on {len(val_dataset)} samples")

train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False, num_workers=0)

--- Fold 1 ---
Training on 621 samples
Validating on 156 samples


In [25]:
# define model & optimizer & loss
model = PaRaMIL().to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Parameter Number: {total_params}")
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr,
                             weight_decay=decay)
criterion = NLLSurvLoss(alpha=0.0)

Parameter Number: 3945670


In [26]:
all_train_losses, all_train_indexes, all_val_losses, all_val_indexes = [], [], [], []

In [28]:
# training loop
for epoch in range(start_epoch, max_epochs):
    random.seed(epoch)
    train_loss, val_loss = 0., 0.
    model.train()

    all_risk_scores = np.zeros((len(train_loader)))
    all_censorships = np.zeros((len(train_loader)))
    all_event_times = np.zeros((len(train_loader)))

    for batch_idx, data in tqdm(enumerate(train_loader)):
        patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn, \
            surv_discrete, surv_time, censor, patient_name = data

        data_WSI = patho.to(device)
        data_omic1 = radio_shape.type(torch.FloatTensor).to(device)
        data_omic2 = radio_order.type(torch.FloatTensor).to(device)
        data_omic3 = radio_texture.type(torch.FloatTensor).to(device)
        data_omic4 = radio_log_sigma.type(torch.FloatTensor).to(device)
        data_omic5 = radio_wavelet.type(torch.FloatTensor).to(device)
        data_omic6 = radio_cnn.type(torch.FloatTensor).to(device)
        surv_discrete = surv_discrete.type(torch.LongTensor).to(device)
        censor = censor.type(torch.FloatTensor).to(device)

        hazards, S, _, _ = model(x_path=data_WSI, x_omic1=data_omic1, x_omic2=data_omic2,
                                 x_omic3=data_omic3, x_omic4=data_omic4, x_omic5=data_omic5,
                                 x_omic6=data_omic6)

        # calculate the loss
        loss = criterion(hazards=hazards, S=S, Y=surv_discrete, c=censor)  # nll_loss
        risk = -torch.sum(S, dim=1).detach().cpu().numpy()
        all_risk_scores[batch_idx] = risk
        all_censorships[batch_idx] = censor.item()
        all_event_times[batch_idx] = surv_time.item()

        loss_value = loss.item()
        train_loss += loss_value

        loss = loss / gc
        loss.backward()
        if (batch_idx + 1) % gc == 0:
            optimizer.step()
            optimizer.zero_grad()

    c_index_train = concordance_index_censored((1 - all_censorships).astype(bool),
                                               all_event_times, all_risk_scores,
                                               tied_tol=1e-08)[0]
    epoch_train_loss = train_loss / len(train_loader)
    all_train_losses.append(epoch_train_loss)
    all_train_indexes.append(c_index_train)
    print(f"[Train] Epoch: {epoch}, overall loss {epoch_train_loss:.4f}, c-index {c_index_train:.4f}.")

    # validation stage
    with torch.no_grad():
        model.eval()
        all_risk_scores_val = np.zeros((len(val_loader)))
        all_censorships_val = np.zeros((len(val_loader)))
        all_event_times_val = np.zeros((len(val_loader)))

        for batch_idx, data in tqdm(enumerate(val_loader)):
            patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn, \
            surv_discrete, surv_time, censor, patient_name = data

            data_WSI = patho.to(device)
            data_omic1 = radio_shape.type(torch.FloatTensor).to(device)
            data_omic2 = radio_order.type(torch.FloatTensor).to(device)
            data_omic3 = radio_texture.type(torch.FloatTensor).to(device)
            data_omic4 = radio_log_sigma.type(torch.FloatTensor).to(device)
            data_omic5 = radio_wavelet.type(torch.FloatTensor).to(device)
            data_omic6 = radio_cnn.type(torch.FloatTensor).to(device)
            surv_discrete = surv_discrete.type(torch.LongTensor).to(device)
            censor = censor.type(torch.FloatTensor).to(device)

            hazards, S, _, _ = model(x_path=data_WSI, x_omic1=data_omic1, x_omic2=data_omic2,
                                     x_omic3=data_omic3, x_omic4=data_omic4, x_omic5=data_omic5,
                                     x_omic6=data_omic6)

            # calculate the loss
            loss = criterion(hazards=hazards, S=S, Y=surv_discrete, c=censor)  # nll_loss

            risk = -torch.sum(S, dim=1).detach().cpu().numpy()
            all_risk_scores_val[batch_idx] = risk
            all_censorships_val[batch_idx] = censor.item()
            all_event_times_val[batch_idx] = surv_time.item()

            loss_value = loss.item()
            val_loss += loss_value

            del patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn

        c_index_val = concordance_index_censored((1 - all_censorships_val).astype(bool),
                                                 all_event_times_val, all_risk_scores_val,
                                                 tied_tol=1e-08)[0]
        if c_index_val > best_c_index_val:
            best_c_index_val = c_index_val
            
        epoch_val_loss = val_loss / len(val_loader)
        all_val_losses.append(epoch_val_loss)
        all_val_indexes.append(c_index_val)
        print(f"[Validation] Epoch: {epoch}, overall loss {epoch_val_loss:.4f}, c-index {c_index_val:.4f}, "
              f"best c-index {best_c_index_val:.4f}.")
    
    if epoch > 20:
        if val_loss + min_delta < min_loss_val:
            min_loss_val = val_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered.")
                break

621it [01:59,  5.19it/s]


[Train] Epoch: 0, overall loss 0.5179, c-index 0.4959.


156it [00:28,  5.45it/s]


[Validation] Epoch: 0, overall loss 0.5083, c-index 0.7042, best c-index 0.7042.


621it [02:00,  5.15it/s]


...

[Validation] Epoch: 31, overall loss 0.4959, c-index 0.7783, best c-index 0.8120.


621it [01:51,  5.57it/s]


[Train] Epoch: 32, overall loss 0.3259, c-index 0.9009.


156it [00:28,  5.47it/s]

[Validation] Epoch: 32, overall loss 0.5604, c-index 0.7457, best c-index 0.8120.
Early stopping triggered.
